In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [2]:
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(["INDUS","AGE"],axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=["PRICES"])
features

,CRIM,ZN,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,0.0,0.538,6.575,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,0.0,0.469,6.421,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,0.0,0.469,7.185,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,0.0,0.458,6.998,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,0.0,0.458,7.147,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,0.0,0.573,6.593,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,0.0,0.573,6.120,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,0.0,0.573,6.976,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,0.0,0.573,6.794,2.3889,1.0,273.0,21.0,393.45,6.48


In [3]:
property_stats = np.ndarray(shape=[1,11])
property_stats = features.mean().values.reshape(1,11)
property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [4]:
RM_IDX=4
PTRATIO_IDX=8
CHAS_IDX=2

In [5]:
regr = LinearRegression().fit(features, target)
fitted_values = regr.predict(features)

mse = mean_squared_error(target,fitted_values)
rmse = mean_squared_error(target,fitted_values,squared=False)

In [6]:
def get_log_estimate(nr_rooms,
                    students_per_classrom,
                    next_to_rive=False,
                    high_confidence=True):
    
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classrom
    
    if next_to_rive:
        property_stats[0][CHAS_IDX]=1
    else:
        property_stats[0][CHAS_IDX]=0
        
    log_estimate=regr.predict(property_stats)[0][0]
    
    if high_confidence:
        upper_bound = log_estimate + 2*rmse
        lower_bound = log_estimate - 2*rmse
        interval = 95
    else:
        upper_bound = log_estimate + rmse
        lower_bound = log_estimate - rmse 
        interval = 68 
    
    return log_estimate, upper_bound, lower_bound, interval

In [7]:
get_log_estimate(3,20,True)

(2.776758191480398, 3.1517824618746584, 2.4017339210861373, 95)

In [8]:
ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_estimate(9,15,False,False)

In [9]:
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
upper_rond = np.e**upper * 1000 * SCALE_FACTOR
lower_rond = np.e**lower * 1000 * SCALE_FACTOR

print("Precio: $", round(dollar_est,-3))
print("Rango superior: $", round(upper_rond,-3))
print("Rango inferior: $", round(lower_rond,-3))

Precio: $ 827000.0
Rango superior: $ 997000.0
Rango inferior: $ 685000.0


In [15]:
def get_price_estimated(rm, pt, river=False, erange=True):
    
    if rm < 1 or pt < 1:
        print('No es posible, intente con valores reales')
        return
    
    log_est, upper, lower, conf = get_log_estimate(rm,pt,river,erange)    
    
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    upper_rond = np.e**upper * 1000 * SCALE_FACTOR
    lower_rond = np.e**lower * 1000 * SCALE_FACTOR
    
    print("Precio: $", round(dollar_est,-3))
    print("Rango superior: $", round(upper_rond,-3))
    print("Rango inferior: $", round(lower_rond,-3))

In [18]:
get_price_estimated(2,20,True,True)

Precio: $ 404000.0
Rango superior: $ 587000.0
Rango inferior: $ 277000.0


In [19]:
import boston_valuation as val

ModuleNotFoundError: No module named 'boston_valuation'